In [ ]:
# import needed libraries
import sys
from collections import OrderedDict
from matplotlib import pyplot as plt
import numpy as np 
import pandas as pd
# path of local project
folder_path = "/home/jiaxiangyi/Documents/rvesimulator"
sys.path.insert(0, folder_path) 

# import local packages and functions 
import rvesimulator 
from simulator_caller import SimulatorCaller
from rvesimulator.design_of_experiment.samplers import FixNumberSampler
from rvesimulator.simulators.path_generator import PathGenerator
from rvesimulator.microstructures.microstructure_plots import DrawRVE2D

In [ ]:
# create the doe
# define the design space
doe_variables = OrderedDict({"control_points": 7})

# define number of samples
num_points = 1
# define the information of outputs
name_outputs = ["PK2", "Green_strain", "ALLPD"]
doe_sampler = FixNumberSampler()
doe_sampler.sampling(
    num_samples=num_points,
    design_space=doe_variables,
    out_names=name_outputs,
    seed=123456,
)
data = doe_sampler.data
print(data)

In [ ]:
#
loads_path_temp = np.empty([num_points, 1])
loads_path_temp[:] = np.nan
# print(loads_path_temp)
loads_path = pd.DataFrame(loads_path_temp, columns=["loads_path"])
loads_path["loads_path"] = loads_path["loads_path"].astype(object)
# print(loads_path)

# define the path generator
for ii in range(num_points):
    path_generator = PathGenerator(num_control_points=data["samples"].at[ii, "control_points"])   
    loads_path.iloc[ii, 0] = path_generator.quadratic_interpolate()
    path_generator.plot_path()

# add the loads path to the samples
data["samples"] = pd.concat(
    [data["samples"], loads_path], axis=1, join="inner"
)
print(data)

## Benchmark 1 



In [ ]:
simulation_wrapper = SimulatorCaller()
simulation_wrapper.update_sim_info(loads=[0.02, 0.02, 0.02], mesh_partition=100, print_info=True)
# plot the micro_structure 
location_info = simulation_wrapper.sim_info["location_information"] 
location_info = np.array(location_info) 
DrawRVE2D.cricle_inclusion_plot(circle_position=location_info,
radius=simulation_wrapper.sim_info["radius"], 
len_start=simulation_wrapper.sim_info["len_start"],
len_end = simulation_wrapper.sim_info["len_end"], 
wid_start=simulation_wrapper.sim_info["wid_start"], 
wid_end = simulation_wrapper.sim_info["wid_end"],
vol_frac=0.02 
 )

In [ ]:
result_benchmark_1 = simulation_wrapper.run_simulation(data=data)

In [ ]:
# simulation results for the first sample

results = result_benchmark_1["responses"]
PK2 = results["PK2"].iloc[0]
Green_strain = results["Green_strain"].iloc[0]
ALLPD = results["ALLPD"].iloc[0]

pparam = dict(xlabel='time step ', ylabel="$\epsilon_{xx}$")
fig, ax = plt.subplots(3,1)
ax[0].plot(Green_strain[:, 0, 0], label="xx") 
ax[0].legend()
ax[0].set(**pparam)
pparam = dict(xlabel='time step ', ylabel="$\epsilon_{xy}$")
ax[1].plot(Green_strain[:, 0, 1], label="xy")
ax[1].legend()
ax[1].set(**pparam)
pparam = dict(xlabel='time step ', ylabel="$\epsilon_{yy}$")
ax[2].plot(Green_strain[:, 1, 1], label="yy") 
ax[2].legend()
ax[2].set(**pparam)
fig.suptitle('strain results')
plt.savefig('strain.png', dpi=300)

pparam = dict(xlabel='time step ', ylabel="$\sigma_{xx}$ (MPa)")
fig, ax = plt.subplots(3,1)
# lt.figure()
# ax.plot(Green_strain[:,0,0],  PK2[:,0,0], label = 'PK2')
ax[0].plot(PK2[:, 0, 0], label="xx") 
ax[0].legend()
ax[0].set(**pparam)
pparam = dict(xlabel='time step ', ylabel="$\sigma_{xy}$ (MPa)")
ax[1].plot(PK2[:, 1, 0], label="xy")
ax[1].legend()
ax[1].set(**pparam)
pparam = dict(xlabel='time step ', ylabel="$\sigma_{yy}$ (MPa)")
ax[2].plot(PK2[:, 1, 1], label="yy") 
ax[2].legend()
ax[2].set(**pparam)
fig.suptitle('stress results')
# plt.show()
plt.savefig('stress.png', dpi=300)


pparam = dict(xlabel='time step ', ylabel="plastic energy")
fig, ax = plt.subplots()
ax.plot(ALLPD[:,0], ALLPD[:,1], label="plastic energy") 
ax.legend()
ax.set(**pparam)
plt.savefig('energy.png', dpi=300)

In [ ]:
simulation_wrapper.update_sim_info(loads=[0.02, 0.02, 0.02], mesh_partition=100, benchmark="benchmark_2", print_info=True)
# plot the micro_structure 
location_info = simulation_wrapper.sim_info["location_information"] 
location_info = np.array(location_info) 
DrawRVE2D.cricle_inclusion_plot(circle_position=location_info,
radius=simulation_wrapper.sim_info["radius"], 
len_start=simulation_wrapper.sim_info["len_start"],
len_end = simulation_wrapper.sim_info["len_end"], 
wid_start=simulation_wrapper.sim_info["wid_start"], 
wid_end = simulation_wrapper.sim_info["wid_end"],
vol_frac=0.02 
 )


In [ ]:
result_benchmark_2 = simulation_wrapper.run_simulation(data=data)

In [ ]:
results = result_benchmark_2["responses"]
PK2 = results["PK2"].iloc[0]
Green_strain = results["Green_strain"].iloc[0]
ALLPD = results["ALLPD"].iloc[0]

pparam = dict(xlabel='time step ', ylabel="$\epsilon_{xx}$")
fig, ax = plt.subplots(3,1)
ax[0].plot(Green_strain[:, 0, 0], label="xx") 
ax[0].legend()
ax[0].set(**pparam)
pparam = dict(xlabel='time step ', ylabel="$\epsilon_{xy}$")
ax[1].plot(Green_strain[:, 0, 1], label="xy")
ax[1].legend()
ax[1].set(**pparam)
pparam = dict(xlabel='time step ', ylabel="$\epsilon_{yy}$")
ax[2].plot(Green_strain[:, 1, 1], label="yy") 
ax[2].legend()
ax[2].set(**pparam)
fig.suptitle('strain results')
plt.savefig('strain.png', dpi=300)

pparam = dict(xlabel='time step ', ylabel="$\sigma_{xx}$ (MPa)")
fig, ax = plt.subplots(3,1)
# lt.figure()
# ax.plot(Green_strain[:,0,0],  PK2[:,0,0], label = 'PK2')
ax[0].plot(PK2[:, 0, 0], label="xx") 
ax[0].legend()
ax[0].set(**pparam)
pparam = dict(xlabel='time step ', ylabel="$\sigma_{xy}$ (MPa)")
ax[1].plot(PK2[:, 1, 0], label="xy")
ax[1].legend()
ax[1].set(**pparam)
pparam = dict(xlabel='time step ', ylabel="$\sigma_{yy}$ (MPa)")
ax[2].plot(PK2[:, 1, 1], label="yy") 
ax[2].legend()
ax[2].set(**pparam)
fig.suptitle('stress results')
# plt.show()
plt.savefig('stress.png', dpi=300)


pparam = dict(xlabel='time step ', ylabel="plastic energy")
fig, ax = plt.subplots()
ax.plot(ALLPD[:,0], ALLPD[:,1], label="plastic energy") 
ax.legend()
ax.set(**pparam)
plt.savefig('energy.png', dpi=300)

In [ ]:
simulation_wrapper.update_sim_info(loads=[0.02, 0.02, 0.02], mesh_partition=100, benchmark="benchmark_3", print_info=True)
# plot the micro_structure 
location_info = simulation_wrapper.sim_info["location_information"] 
location_info = np.array(location_info) 
DrawRVE2D.cricle_inclusion_plot(circle_position=location_info,
radius=simulation_wrapper.sim_info["radius"], 
len_start=simulation_wrapper.sim_info["len_start"],
len_end = simulation_wrapper.sim_info["len_end"], 
wid_start=simulation_wrapper.sim_info["wid_start"], 
wid_end = simulation_wrapper.sim_info["wid_end"],
vol_frac=0.02 
 )


In [ ]:
result_benchmark_3 = simulation_wrapper.run_simulation(data=data)

In [ ]:
results = result_benchmark_3["responses"]
PK2 = results["PK2"].iloc[0]
Green_strain = results["Green_strain"].iloc[0]
ALLPD = results["ALLPD"].iloc[0]

pparam = dict(xlabel='time step ', ylabel="$\epsilon_{xx}$")
fig, ax = plt.subplots(3,1)
ax[0].plot(Green_strain[:, 0, 0], label="xx") 
ax[0].legend()
ax[0].set(**pparam)
pparam = dict(xlabel='time step ', ylabel="$\epsilon_{xy}$")
ax[1].plot(Green_strain[:, 0, 1], label="xy")
ax[1].legend()
ax[1].set(**pparam)
pparam = dict(xlabel='time step ', ylabel="$\epsilon_{yy}$")
ax[2].plot(Green_strain[:, 1, 1], label="yy") 
ax[2].legend()
ax[2].set(**pparam)
fig.suptitle('strain results')
plt.savefig('strain.png', dpi=300)

pparam = dict(xlabel='time step ', ylabel="$\sigma_{xx}$ (MPa)")
fig, ax = plt.subplots(3,1)
# lt.figure()
# ax.plot(Green_strain[:,0,0],  PK2[:,0,0], label = 'PK2')
ax[0].plot(PK2[:, 0, 0], label="xx") 
ax[0].legend()
ax[0].set(**pparam)
pparam = dict(xlabel='time step ', ylabel="$\sigma_{xy}$ (MPa)")
ax[1].plot(PK2[:, 1, 0], label="xy")
ax[1].legend()
ax[1].set(**pparam)
pparam = dict(xlabel='time step ', ylabel="$\sigma_{yy}$ (MPa)")
ax[2].plot(PK2[:, 1, 1], label="yy") 
ax[2].legend()
ax[2].set(**pparam)
fig.suptitle('stress results')
# plt.show()
plt.savefig('stress.png', dpi=300)


pparam = dict(xlabel='time step ', ylabel="plastic energy")
fig, ax = plt.subplots()
ax.plot(ALLPD[:,0], ALLPD[:,1], label="plastic energy") 
ax.legend()
ax.set(**pparam)
plt.savefig('energy.png', dpi=300)

# plot all results together 

In [ ]:
results_1 = result_benchmark_1["responses"]
results_2 = result_benchmark_2["responses"]
results_3 = result_benchmark_3["responses"]

# result of benchmark1
PK2_1 = results_1["PK2"].iloc[0]
Green_strain_1 = results_1["Green_strain"].iloc[0]
ALLPD_1 = results_1["ALLPD"].iloc[0]
# result of benchmark2
PK2_2 = results_2["PK2"].iloc[0]
Green_strain_2 = results_2["Green_strain"].iloc[0]
ALLPD_2 = results_2["ALLPD"].iloc[0]

#result of benchmark3
PK2_3 = results_3["PK2"].iloc[0]
Green_strain_3 = results_3["Green_strain"].iloc[0]
ALLPD_3 = results_3["ALLPD"].iloc[0]


pparam = dict(xlabel='time step ', ylabel="$\epsilon_{xx}$")
fig, ax = plt.subplots(3,1)
ax[0].plot(Green_strain_1[:, 0, 0], label="benchmark 1") 
ax[0].plot(Green_strain_2[:, 0, 0], label="benchmark 2") 
ax[0].plot(Green_strain_3[:, 0, 0], label="benchmark 3") 
ax[0].legend()
ax[0].set(**pparam)
pparam = dict(xlabel='time step ', ylabel="$\epsilon_{xy}$")
ax[1].plot(Green_strain_1[:, 0, 1], label="benchmark 1")
ax[1].plot(Green_strain_2[:, 0, 1], label="benchmarl 2")
ax[1].plot(Green_strain_3[:, 0, 1], label="benchmark 3")
ax[1].legend()
ax[1].set(**pparam)
pparam = dict(xlabel='time step ', ylabel="$\epsilon_{yy}$")
ax[2].plot(Green_strain_1[:, 1, 1], label="benchmark 1") 
ax[2].plot(Green_strain_2[:, 1, 1], label="benchmark 2") 
ax[2].plot(Green_strain_3[:, 1, 1], label="benchmark 3") 
ax[2].legend()
ax[2].set(**pparam)
fig.suptitle('strain results')
plt.savefig('strain.png', dpi=300)

pparam = dict(xlabel='time step ', ylabel="$\sigma_{xx}$ (MPa)")
fig, ax = plt.subplots(3,1)
ax[0].plot(PK2_1[:, 0, 0], label="benchmark 1") 
ax[0].plot(PK2_2[:, 0, 0], label="benchmark 2") 
ax[0].plot(PK2_3[:, 0, 0], label="benchmark 3") 
ax[0].legend()
ax[0].set(**pparam)
pparam = dict(xlabel='time step ', ylabel="$\sigma_{xy}$ (MPa)")
ax[1].plot(PK2_1[:, 1, 0], label="benchmark 1")
ax[1].plot(PK2_2[:, 1, 0], label="benchmark 2")
ax[1].plot(PK2_3[:, 1, 0], label="benchmark 3")
ax[1].legend()
ax[1].set(**pparam)
pparam = dict(xlabel='time step ', ylabel="$\sigma_{yy}$ (MPa)")
ax[2].plot(PK2_1[:, 1, 1], label="benchmark 1") 
ax[2].plot(PK2_2[:, 1, 1], label="benchmark 2") 
ax[2].plot(PK2_3[:, 1, 1], label="benchmark 3") 
ax[2].legend()
ax[2].set(**pparam)
fig.suptitle('stress results')
# plt.show()
plt.savefig('stress.png', dpi=300)


pparam = dict(xlabel='time step ', ylabel="plastic energy")
fig, ax = plt.subplots()
ax.plot(ALLPD_1[:,0], ALLPD[:,1], label="benchmark 1") 
ax.plot(ALLPD_2[:,0], ALLPD[:,1], label="benchmark 2") 
ax.plot(ALLPD_3[:,0], ALLPD[:,1], label="benchmakr 3") 
ax.legend()
ax.set(**pparam)
plt.savefig('energy.png', dpi=300)